In [907]:
import requests as r
import json
import pandas as pd

In [908]:
base_url = 'https://form-service.akvotest.org/api/form/'

In [909]:
forms = [{
    "id":1699353915355,
    "name": "Households",
},{
    "id": 1701757876668,
    "name": "Wash in Schools",
},{
    "id": 1699354006503,
    "name": "Community",
}]

In [910]:
def question_mapping(row):
    qs = pd.DataFrame(row[0]).rename(columns={'name': 'question', 'option': 'options'})
    qs = qs.where(pd.notnull(qs), None)
    return qs.to_dict('records')

In [911]:
def generate_form(data):
    df = pd.json_normalize(data)
    df['type'] = 1
    df = df.rename(columns={'name': 'form', 'question_group': 'question_groups'})
    df2 = pd.DataFrame(df['question_groups'][0])
    df2 = df2.rename(columns={'name': 'question_group', 'question': 'questions'})
    df2['questions'] = pd.DataFrame(df2['questions']).apply(question_mapping, axis=1)
    qg = df2.to_dict('records')
    df['question_groups'] =  df['question_groups'].apply(lambda x: qg)
    j = df.to_dict('records')[0]
    with open(f"../../../backend/source/forms-2023/{form_id}.prod.json", 'w') as f:
        json.dump(j, f)

In [912]:
for form in forms:
    form_id = form['id']
    req = r.get(f"{base_url}{form_id}")
    generate_form(data=req.json())